<a href="https://colab.research.google.com/github/chrisoyer/thinkful_notes/blob/master/Unit_4_Spark/Amazon_Kindle_Reviews__Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description:
Challenge: Amazon Reviews Sentiment Analysis

### Initialization

In [0]:
#environment
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [2]:
# Install spark-related depdencies for Python
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 90kB/s 
     |████████████████████████████████| 204kB 42.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=91c4769138bd1722ff797640de60ed8923d09cd5120c1a6ec210c4068e42f358
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
source_url = r"http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz"

I will be using gDrive file location instead of downloading the source file.

In [35]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
local_souce_file = r"/content/gdrive/My Drive/thinkful/colab_datasets/reviews_Grocery_and_Gourmet_Food_5.json.gz"
SPARK_URL = "local[*]"
APP_NAME  = "amazon_food_reviews"

In [0]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
import pyspark.sql.functions as F

In [0]:
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
# not working# !gzip -dkc $local_source_file > /content/reviews
reviews = r'/content/gdrive/My Drive/thinkful/colab_datasets/reviews_Grocery_and_Gourmet_Food_5.json'
reviews_df = spark.read.options(inferschema = "true").json(reviews)

In [25]:
!file $local_source_file

/bin/bash: file: command not found


In [45]:
reviews_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [46]:
reviews_df.show(5)

+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|   reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+
|616719923X| [0, 0]|    4.0|Just another flav...| 06 1, 2013|A1VEELTKS8NLZB|Amazon Customer|          Good Taste|    1370044800|
|616719923X| [0, 1]|    3.0|I bought this on ...|05 19, 2014|A14R9XMZVJ6INB|        amf0001|3.5 stars,  sadly...|    1400457600|
|616719923X| [3, 4]|    4.0|Really good. Grea...| 10 8, 2013|A27IQHDZFQFNGG|        Caitlin|                Yum!|    1381190400|
|616719923X| [0, 0]|    5.0|I had never had i...|05 20, 2013|A31QY5TASILE89|   DebraDownSth|Unexpected flavor...|    1369008000|
|616719923X| [1, 2]|    4.0|I've been looking...|05 26, 2013|A2LWK003FFMCI5|       Diana X.|Not a

In [49]:
!pip install spark-nlp
import sparknlp
spartnlp.start()
pipeline = PretrainedPipeline(name='analyze_sentiment', lang='en')